In [1]:
import json
from pprint import pprint
import sys
import unicodedata

import fiona
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string, from_epsg
from fiona.transform import transform_geom
from pyproj import Proj

from madmex.models import Region, Country
from madmex.util.spatial import feature_transform

In [2]:
#name_admin = 'MEX' or column_name = 'AGUACATE_DIANA'
name_admin = 'mexico_contorno'
#column_name = 'NAME_1' or column_name = ''
column_name = ''
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/MEX_adm_shp/MEX_adm1.shp' 
#or 
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'
shapefile = '/LUSTRE/MADMEX/shapefiles/mexico_contorno.shp'

In [3]:
with fiona.open(shapefile,
                encoding='utf-8') as src:
    fc = list(src)
    src_crs = src.crs
    to_string_crs = to_string(src_crs)
    proj_crs = Proj(src.crs)
    if not proj_crs.crs.is_geographic:
        fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
    else:
        fc_proj = fc

In [4]:
[feat['geometry']['type'] for feat in fc_proj][0:4]

['Polygon', 'Polygon', 'Polygon', 'Polygon']

In [6]:
shape_list = [shape(feat['geometry']) for feat in fc_proj]

In [7]:
country_shape = cascaded_union(shape_list)

In [8]:
country, _ = Country.objects.get_or_create(the_geom=GEOSGeometry(country_shape.wkt),
                                       name=name_admin)

In [9]:
# Ingest features
for k in range(0,len(fc_proj)):
    shape = shape_list[k]
    geom = GEOSGeometry(shape.wkt,4326)
    
    if not column_name: #needs to generate name for every entry
        name = name_admin + '_%s'
        name_feature = name % fc_proj[k]['id']
    else:
        name_feature = fc_proj[k]['properties'][column_name].replace(" ", "_")
        name_feature_normalized = unicodedata.normalize('NFKD', name_feature).encode('ASCII', 'ignore').decode('utf-8')
        name_feature = name_admin + '_' + name_feature_normalized
    _ = Region.objects.get_or_create(name=name_feature,
                                     the_geom=geom,
                                     country=country)
    print(name_feature)

mexico_contorno_0
mexico_contorno_1
mexico_contorno_2
mexico_contorno_3
mexico_contorno_4
mexico_contorno_5
mexico_contorno_6
mexico_contorno_7
mexico_contorno_8
mexico_contorno_9
mexico_contorno_10
mexico_contorno_11
mexico_contorno_12
mexico_contorno_13
mexico_contorno_14
mexico_contorno_15
mexico_contorno_16
mexico_contorno_17
mexico_contorno_18
mexico_contorno_19
mexico_contorno_20
mexico_contorno_21
mexico_contorno_22
mexico_contorno_23
mexico_contorno_24
mexico_contorno_25
mexico_contorno_26
mexico_contorno_27
mexico_contorno_28
mexico_contorno_29
mexico_contorno_30
mexico_contorno_31
mexico_contorno_32
mexico_contorno_33
mexico_contorno_34
mexico_contorno_35
mexico_contorno_36
mexico_contorno_37
mexico_contorno_38
mexico_contorno_39
mexico_contorno_40
mexico_contorno_41
mexico_contorno_42
mexico_contorno_43
mexico_contorno_44
mexico_contorno_45
mexico_contorno_46
mexico_contorno_47
mexico_contorno_48
mexico_contorno_49
mexico_contorno_50
mexico_contorno_51
mexico_contorno_52
mex

In [10]:
region = 'mexico_contorno_0'
region_geom = Region.objects.get(name=region).the_geom
region_geom

In [11]:
region_geom.wkt

'POLYGON ((-96.49071411955359 15.66210837112926, -96.49461923404333 15.65827474614957, -96.49469357555982 15.65847700800528, -96.4948787840919 15.65876994819001, -96.49482210472348 15.65895853862635, -96.49496384172483 15.65910683786333, -96.49496006805767 15.65920204678336, -96.49490433111426 15.65936683538231, -96.4950989752619 15.65942175292376, -96.49526376744305 15.65943734516841, -96.49540476760711 15.65948434539576, -96.49540476745879 15.65971934535632, -96.49563976689426 15.65976634518888, -96.49559276696471 15.66004834582292, -96.49592176685142 15.66037734562934, -96.49545176677719 15.66051834587904, -96.4954517672806 15.66084734556245, -96.49592176731736 15.66098834533333, -96.49592176718866 15.6611293459296, -96.49559276686036 15.6612703452356, -96.49545176678882 15.66164634548334, -96.49531076720055 15.66159934517641, -96.49517186431065 15.66184242571655, -96.49484398815704 15.66150996756532, -96.49469281307162 15.66159969225406, -96.49468620866269 15.66176630992119, -96.49

In [15]:
country = 'mexico_contorno'
country_geom = Country.objects.get(name=country).the_geom
country_geom